In [1]:
%load_ext autoreload
%autoreload 2
import sys 
import plotly.express as px
import plotly.graph_objects as go
import re
import time

from lib import utils
from lib.slurm import SlurmClient

# Slurm Queue

In [2]:
sl = SlurmClient()
q = sl.get_queue_summary()

print("Total:", q.NODES.sum())
print("Total Running: ", q[q.ST=='R'].NODES.sum())
q

Total: 501
Total Running:  437


,ACCOUNT,ST,NODES
0,all,R,17
1,ar-ai-hipri,PD,64
2,ar-ai-hipri,R,331
3,ar-ai-midpri,R,34
4,ar-ai-research-interns,R,8
5,ar-ai-voice-hipri,R,47


In [18]:
q_ar = sl.get_queue()
q_ar = q_ar[(q_ar.ACCOUNT == 'ar-ai-hipri') & (q_ar.ST == 'R')]
fig = px.pie(q_ar, values='NODES', names='USER')
fig.update_traces(textinfo='label+value')

# Evict

In [27]:
sl.get_job_launch_cmd(20484)

cd fsx_0/user/shiyudong/openCLIPMeta
sbatch scripts/aws/aws_e14_gila_finetune_ocr_i448.sh


In [13]:
# evic interns
while True:
    interns = ['yfzhuang', 'jiuhai', 'imzyc']
    sl = SlurmClient()
    q = sl.get_queue()

    q = q[q.USER.isin(interns)]
    q = q[~q.ACCOUNT.isin(['ar-ai-research-interns', 'ar-ai-voice-hipri'])]

    print(q[['JOBID', 'ST', 'ACCOUNT', 'USER', 'NODES']])

    for j in list(q.JOBID):
        print(f"Cancelling job {j}")
        utils.get_bash_output(f"scancel {j}")

    time.sleep(2)

    JOBID ST       ACCOUNT      USER  NODES
70  25987  R  ar-ai-midpri  yfzhuang     16
Cancelling job 25987
   JOBID  ST       ACCOUNT      USER  NODES
5  25987  CG  ar-ai-midpri  yfzhuang     16
Cancelling job 25987


KeyboardInterrupt: 

In [9]:
q = sl.get_queue()
q = q[q.ACCOUNT.isin(['ar-ai-midpri'])]

print(q[['JOBID', 'ST', 'ACCOUNT', 'USER', 'NODES']])
for j in list(q.JOBID):
    print(f"Cancelling job {j}")
    utils.get_bash_output(f"scancel {j}")

     JOBID ST       ACCOUNT      USER  NODES
397  24837  R  ar-ai-midpri  tomfujii      1
Cancelling job 24837


In [73]:
# evic me
q = sl.get_recent_queue('ar-ai-hipri')
q = q[q.USER == 'tranx']
# q = q[q.ST == 'RH']
q
# for _, job in q.iterrows():
# for j in q.JOBID:
#     if j != '12811':
#         utils.get_bash_output(f"scancel {j}")
#         print(f"Cancel {j}")

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
42,14330,ar-ai-hipri,s2_i18n,tranx,R,2024-08-22T17:49:45,1-00:54:45,64,89685
41,14332,ar-ai-hipri,s2_exp28,tranx,R,2024-08-22T17:23:50,1-01:20:40,128,91240
43,14171,ar-ai-hipri,s2_504,tranx,R,2024-08-22T00:59:10,1-17:45:20,64,150320
44,13932,ar-ai-hipri,bash,tranx,R,2024-08-21T19:21:55,1-23:22:35,1,170555


In [54]:
q = sl.get_queue_summary()
running = {}
for _, row in q.iterrows():
    if row.ST == 'R':
        running[row.ACCOUNT] = row.NODES
print(running)

quota = {'ar-ai-research-interns': 8,
         'ar-ai-voice-hipri': 50, 'ar-ai-hipri': 400}

evict_ids = {}
for acc in running:
    if acc not in quota:  # quota = 0
        print(acc)

{'all': 17, 'ar-ai-hipri': 274, 'ar-ai-midpri': 2, 'ar-ai-research-interns': 3, 'ar-ai-voice-hipri': 53}
all
ar-ai-midpri


In [30]:
q_summary = q.groupby(['ACCOUNT', 'ST']).aggregate(
    {'NODES': 'sum'}).reset_index()
q_summary

,ACCOUNT,ST,NODES
0,all,R,17
1,ar-ai-hipri,R,280
2,ar-ai-midpri,R,2
3,ar-ai-research-interns,R,3
4,ar-ai-voice-hipri,PD,94
5,ar-ai-voice-hipri,R,53


# Pretrain

In [48]:
MM9_CONF_DIR = "/fsx_0/user/tranx/experiments/llm_mm_aligner/stage1_mm9"
envvar = {
    "NUM": 7,
    "JSON_CONFIG": f"fbl_pretrain_MM9_70B_Llama31_336px_2nodes.json",
    # "CONDA_ENV": "aligner_v7",
}

sbatch_base = "/fsx_0/user/tranx/experiments/llm_mm_aligner/base_pretrain_MM9_70B_sbatch.sh"
sbatch_overwrite = {
    "job-name": "test",
    "nodes": 2,
    "ntasks": 2
}

MM9_CONF_DIR = "/fsx_0/user/tranx/experiments/llm_mm_aligner/stage1_mm9"
JSON_CONFIG = f"{MM9_CONF_DIR}/xxxfbl_pretrain_MM9_70B_Llama31_336px_2nodes.json"
ALIGNER_PARENT_DIR = "/fsx_0/user/tranx"

envvar_string = f"{JSON_CONFIG} {ALIGNER_PARENT_DIR}"

sbatch_vars_string = []
for k, v in sbatch_overwrite.items():
    sbatch_vars_string.append(f"--{k}={v}")
sbatch_vars_string = ' '.join(sbatch_vars_string)

# cmd = f'sbatch --parsable {sbatch_vars_string} {sbatch_base} {envvar_string}'
cmd = f'sbatch --parsable {sbatch_vars_string} {sbatch_base} a b'
print(cmd)

job_id = utils.get_bash_output(cmd, print_output=True)
job_id = int(job_id)

sbatch --parsable --job-name=test --nodes=2 --ntasks=2 /fsx_0/user/tranx/experiments/llm_mm_aligner/base_pretrain_MM9_70B_sbatch.sh a b
9659



In [29]:
!tail -f /fsx_0/user/tranx/output/slurm_logs/output_9402.txt

/var/spool/slurmd/job09402/slurm_script: line 23: conda: command not found
/var/spool/slurmd/job09402/slurm_script: line 24: conda: command not found
Using conda environment:
Error: CONDA_DEFAULT_ENV is not set to aligner_v7
^C


# Test trains

# Stage 2